# Assignment 2: Milestone I Natural Language Processing
## Task 2&3
#### Student Name: XXXX XXXX
#### Student ID: 000000

Date: XXXX

Version: 1.0

Environment: Python 3 and Jupyter notebook

Libraries used: please include all the libraries you used in your assignment, e.g.,:
* pandas
* re
* numpy

## Introduction
You should give a brief information of this assessment task here.

<span style="color: red"> Note that this is a sample notebook only. You will need to fill in the proper markdown and code blocks. You might also want to make necessary changes to the structure to meet your own needs. Note also that any generic comments written in this notebook are to be removed and replace with your own words.</span>

## Importing libraries 

In [5]:
# Code to import libraries as you need in this assessment, e.g.,
import nltk
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

## Task 2. Generating Feature Representations for Job Advertisement Descriptions

Read the vocab.txt file

In [ ]:
# Read the vocab.txt file
with open('vocab.txt', 'r') as file:
    lines = file.read().splitlines()

# Parse the file and create a list of words
words = [line.split(':')[0] for line in lines]

### Count Vector

In [6]:
# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the data
count_vector = vectorizer.fit_transform(words)

# Convert the count_vector to an array
count_vector_array = count_vector.toarray()
print(count_vector_array)

[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]]


### TF-IDF Vector

### Unweighted Vector

### Saving outputs
Save the count vector representation as per spectification.
- count_vectors.txt

In [ ]:
# code to save output data...

## Task 3. Job Advertisement Classification

...... Sections and code blocks on buidling classification models based on different document feature represetations. 
Detailed comparsions and evaluations on different models to answer each question as per specification. 

<span style="color: red"> You might have complex notebook structure in this section, please feel free to create your own notebook structure. </span>

In [ ]:
# Code to perform the task...


## Summary
Give a short summary and anything you would like to talk about the assessment tasks here.

## Couple of notes for all code blocks in this notebook
- please provide proper comment on your code
- Please re-start and run all cells to make sure codes are runable and include your output in the submission.   
<span style="color: red"> This markdown block can be removed once the task is completed. </span>